In [ ]:
import matplotlib.pyplot as plt
from astropy.table import Table
import numpy as np
import arya

import astropy.units as u
from astropy.nddata import CCDData
from astropy.nddata import Cutout2D
from astropy.coordinates import SkyCoord

import tomllib

In [ ]:
import sys
sys.path.append("..")
sys.path.append("../../imaging/")
from phot_utils import to_mag, get_atm_extinction, show_image
from photutils.aperture import CircularAperture


In [ ]:
def read_catalogue(filename):
    return Table.read(filename, hdu=2, format="fits")

In [ ]:
def plot_pixel_cutouts(
    ccd,
    centers_xy,
    cutout_size=10 * u.arcsec,
    aperture_radius=10,
):
    """
    Plot pixel-coordinate cutouts with circular apertures.

    Parameters
    ----------
    ccd : astropy.nddata.CCDData
        Input image (WCS optional but recommended).
    centers_xy : list of (x, y)
        Pixel coordinates of centers (0-indexed).
    cutout_size : Quantity or int
        Size of cutout (square). If Quantity, converted via WCS.
    aperture_radius : float
        Aperture radius in pixels.
    """
    for i, (x, y) in enumerate(centers_xy):
        cutout = Cutout2D(
            data=ccd.data,
            position=(x, y),
            size=cutout_size,
            wcs=ccd.wcs
        )

        # Center of the source *within the cutout*
        # (important if the cutout was clipped at image edges)
        x_c, y_c = cutout.position_cutout

        aperture = CircularAperture((x_c, y_c), r=aperture_radius)

        fig, ax = plt.subplots()
        show_image(cutout.data, fig=fig, ax=ax, log=True)

        aperture.plot(
            ax=ax,
            color='red',
            lw=1.5
        )

        ax.set_title(f'Cutout {i}')
        ax.set_xlabel('x [pix]')
        ax.set_ylabel('y [pix]')

        plt.tight_layout()
        plt.show()

In [ ]:
def plot_sky_cutouts(ccd, centers, size=10*u.arcsec):
    """
    Plot cutouts around sky-coordinate centers.

    Parameters
    ----------
    ccd : astropy.nddata.CCDData
        Input image with WCS.
    centers : list of astropy.coordinates.SkyCoord
        Center positions.
    size : astropy.units.Quantity
        Cutout size (assumed square).
    """
    for i, center in enumerate(centers):
        cutout = Cutout2D(
            data=ccd.data,
            position=center,
            size=size,
            wcs=ccd.wcs
        )

        fig, ax = plt.subplots(
            subplot_kw={'projection': cutout.wcs},
            figsize=(4, 4)
        )
        im = ax.imshow(cutout.data,  cmap='gray')

        # plt.scatter(*ccd.wcs.world_to_pixel(center), color="red")
        plt.scatter(center.ra, center.dec, color="red")
        ax.set_title(f'Cutout {i}')
        ax.set_xlabel('RA')
        ax.set_ylabel('Dec')

        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        plt.tight_layout()
        plt.show()


In [ ]:
filtname = "g"
objname = "yasone2"
objid = "01"
stdid = "11"

filename = f"../{objname}/julen_stack_g_0004/photometry.cat"
imagename = f"../{objname}/julen_stack_g_0004/stacked.fits"
stdname = f"../std1/img_{filtname}_{stdid}/flat_fielded-astrom-zeropoint.toml"


In [ ]:
with open(stdname, "rb") as f:
    std_data = tomllib.load(f)
std_data

In [ ]:
tab = read_catalogue(filename)
img = CCDData.read(imagename)

assert std_data["filter"] == img.header["filter2"]

In [ ]:
atm_ext, atm_ext_err = get_atm_extinction(img.header["airmass"], img.header["filter2"])

In [ ]:
tab

In [ ]:
tab["MAG"] = tab["MAG_AUTO"] + std_data["zeropoint"]

In [ ]:
filt_good = (tab["FLAGS"] == 0) 
filt_good &= tab["MAG"] < 21
filt_good &= tab["MAG"] > 18
filt_good &= tab["ELLIPTICITY"] < 0.2

In [ ]:
plt.figure()
bins = np.linspace(15, 27, 20)
plt.hist(mags, bins)
plt.hist(mags[filt_good], bins)
plt.yscale("log")

In [ ]:
filt_good

In [ ]:
good_cat = tab[filt_good]

In [ ]:
%matplotlib ipympl

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection=img.wcs))

show_image(img, ax=ax, fig=fig, log=True)
plt.scatter(good_cat["XWIN_IMAGE"], good_cat["YWIN_IMAGE"], s=2, facecolor="none", edgecolor="red", lw=0.3)


In [ ]:
coords = SkyCoord(good_cat["ALPHA_J2000"], good_cat["DELTA_J2000"])

In [ ]:
%matplotlib inline

In [ ]:
cens = [a for a in zip(good_cat["XWIN_IMAGE"], good_cat["YWIN_IMAGE"])]

In [ ]:
plt.close()
plt.figure();

In [ ]:
len(cens)

In [ ]:
plt.hist(good_cat["FWHM_WORLD"].to("arcsec"), bins=np.linspace(0, 3, 50) * u.arcsec)

In [ ]:
plot_pixel_cutouts(img, cens)